In [1]:
from nltk.util import pr
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
stemer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kinge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load the dataset into a pandas DataFrame

data = pd.read_csv("twitter_data - twitter_data.csv")
data.tail(10)

# Preprocess the text data (e.g., removing stopwords, punctuation, etc.)
# Perform any necessary cleaning or text preprocessing steps


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
24773,25286,3,1,2,0,1,you niggers cheat on ya gf's? smh....
24774,25287,3,0,3,0,1,you really care bout dis bitch. my dick all in...
24775,25288,3,0,3,0,1,"you worried bout other bitches, you need me for?"
24776,25289,3,3,0,0,0,you're all niggers
24777,25290,3,2,1,0,0,you're such a retard i hope you get type 2 dia...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies
24782,25296,3,0,0,3,2,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...


In [3]:
data["class"].unique()


array([2, 1, 0], dtype=int64)

In [4]:
data["labels"] = data["class"].map({0:"Hate Speech",1:"Offensive Language",2:"No Hate and Offensive language"})
data = data[["tweet","labels"]]

In [5]:
data

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive language
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,Offensive Language
24779,"you've gone and broke the wrong heart baby, an...",No Hate and Offensive language
24780,young buck wanna eat!!.. dat nigguh like I ain...,Offensive Language
24781,youu got wild bitches tellin you lies,Offensive Language


In [6]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\w'
C:\Users\kinge\AppData\Local\Temp\ipykernel_3308\1876373596.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\kinge\AppData\Local\Temp\ipykernel_3308\1876373596.py:4: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\kinge\AppData\Local\Temp\ipykernel_3308\1876373596.py:8: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [7]:
data["tweet"] = data["tweet"].apply(clean)

In [8]:
x = np.array(data["tweet"])
y = np.array(data["labels"])
cv = CountVectorizer()
x = cv.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=42)
x_train

<16604x25540 sparse matrix of type '<class 'numpy.int64'>'
	with 133088 stored elements in Compressed Sparse Row format>

In [9]:
model = DecisionTreeClassifier()
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [10]:
model.score(x_test,y_test)

0.8700330113705832

In [11]:
sample = "so listen to me and kill all black peoples"
data = cv.transform([sample]).toarray()
print(model.predict(data))

['Offensive Language']


In [12]:
import pickle
pickle.dump(model, open("hate_speech_model.pkl", 'wb'))

In [13]:
pickle.dump(cv, open("cv.pkl", 'wb'))